In [1]:
import pandas as pd
import numpy as np
import sklearn.metrics
import os
from sklearn import preprocessing
from sklearn.externals import joblib
import multiprocessing as mp

#import PatternVectorizer and SimpleClassifier
from pattern_classifier import  SimpleClassifier, PatternVectorizer

In [2]:
cls_persistence = 'data/simple_classifier_model.pkl.compressed'
pv_persistence = 'data/pattern_vectorizer.pkl.compressed'

cls = joblib.load(cls_persistence)
pv = joblib.load(pv_persistence)

## Load tweets

In [3]:
file_dir = 'mention_recent/'
col = ['id', 'screen', 'datetime', 'text']
user_tweet_list = [pd.read_csv(file_dir+file, delimiter = '\t', names = col) for file in os.listdir(file_dir)]

## Query Emotion

In [4]:
def emotion_query(user_tweets):
    documentPatternVectors = pv.transform(user_tweets.text)

    # using two emotion, guess the one with the smallest rank
    Y_GUESS_2 = cls.get_top_emotion_ambiguous(documentPatternVectors, ascending=True, n=2)

    Guess_array = np.array(Y_GUESS_2)

    user_tweets['emotion1'] = Guess_array[:,0]
    user_tweets['emotion2'] = Guess_array[:,1]
    user_tweets['ambiguous'] = Guess_array[:,2]
    
    return user_tweets

### Multi-processing

In [5]:
pool = mp.Pool(processes=mp.cpu_count()-1)
multi_res = [pool.apply_async(emotion_query, (user_tweets,)) for user_tweets in user_tweet_list]

emo_user_tweet_list = [res.get() for res in multi_res]

## Query Sentiment 140

In [37]:
import re
import urllib
import json
from datetime import datetime

In [42]:
# function to delete url
def del_url(line):
    return re.sub(r'(\S*(\.com).*)|(https?:\/\/.*)', "", line)
# replace tag
def checktag(line): 
    return re.sub(r'\@|\#', "", line)
# Some special character
def checkSpecial(line):
    return line.replace('♡', 'love ').replace('\"','').replace('“','').replace('”','').replace('…','...').replace('’','\'').replace('•', '').replace('–','-')

def sendto140(line):
        query_string = '{"data": ['
        # username, date, datetime, content 
        query_string += '{"text": "' + checkSpecial(checktag(del_url(str(line).strip()))) + '"},'

        query_string = query_string[:-1] + ']}'
        try:
            data = query_string.encode('utf-8');
            response = urllib.request.urlopen('http://www.sentiment140.com/api/bulkClassifyJson', data) # request to server     
            page = response.read() # get the response     
            # print page # print the result     
            query_result = json.loads((page).decode('utf-8')) # parse the result. The result is in JSON format
            return sentiment_dict[int(query_result["data"][0]["polarity"])]
        except:
            print('\n{} - Fail : {}'.format( datetime.now().strftime("%Y/%m/%d %H:%M:%S"), query_string))
            return sentiment_dict[2]



sentiment_dict = {
                0:-1,
                2: 0,
                4: 1
                }

In [ ]:
for i, emo_user_tweets in enumerate(emo_user_tweet_list):
    print(emo_user_tweets['screen'][0])
    emo_user_tweet_list[i]['sentiment'] = emo_user_tweets.text.apply(lambda t: sendto140(t))

LynneThrasher
jamQueGorb
NikkiLedgerwood

2017/05/06 19:44:30 - Fail : {"data": [{"text": "AndrewCTE ️ "}]}
